In [1]:
pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 6.1MB/s 


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [3]:
pip install eli5

     |████████████████████████████████| 112kB 4.6MB/s 


In [4]:
import nltk
import sklearn_crfsuite
import eli5

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
import spacy
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.3MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=e7987c26d18c4ac2d39185ef2f23df7f9fab9debad518a6ffbb9d76ad9f0e933
  Stored in directory: /tmp/pip-ephem-wheel-cache-_9bd46oz/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [12]:
nlp = spacy.load("en_core_web_sm") 

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('dev.csv')
dev = pd.read_csv('dev.csv')
train.columns = ['id','Sentence','Label']
dev.columns = ['id','Sentence','Label']
test.columns = ['id','Sentence','Label']
train.head()
dev.head()
test.head()

,id,Sentence,Label
0,TR-0,What is here called controlled natural languag...,O O O O B-long I-long I-long O B-short O O O O...
1,TR-1,"In this paper , we introduce CoQA , a Conversa...",O O O O O O B-short O O B-long I-long I-long O...
2,TR-2,"in : Proceedings of SODA 98 , Ninth Annual ACM...",O O O O B-short O O O O B-short O B-short O O ...
3,TR-3,But is the existing HPC I / O stack still suit...,O O O O B-short I-short I-short I-short O O O ...
4,TR-4,"For a fixed and known , we apply a successive ...",O O O O O O O O O B-long I-long I-long O B-sho...


,id,Sentence,Label
0,DEV-0,The second case in which it helps to pose a no...,O O O O O O O O O O O O B-short O O B-short O ...
1,DEV-1,We created such datasets for each kind of Ques...,O O O O O O O O B-long I-long O O O O O O O O ...
2,DEV-2,"Related Work Currently , several researchers s...",O O O O O O O O O O O O O O B-short O O
3,DEV-3,"However , we observed a decrease in the number...",O O O O O O O O O O O O O O O O O O O O O O O ...
4,DEV-4,"Here , DA = direct assessment , RR = relative ...",O O B-short O B-long I-long O B-short O B-long...


,id,Sentence,Label
0,DEV-0,The second case in which it helps to pose a no...,O O O O O O O O O O O O B-short O O B-short O ...
1,DEV-1,We created such datasets for each kind of Ques...,O O O O O O O O B-long I-long O O O O O O O O ...
2,DEV-2,"Related Work Currently , several researchers s...",O O O O O O O O O O O O O O B-short O O
3,DEV-3,"However , we observed a decrease in the number...",O O O O O O O O O O O O O O O O O O O O O O O ...
4,DEV-4,"Here , DA = direct assessment , RR = relative ...",O O B-short O B-long I-long O B-short O B-long...


In [16]:
train['Sentence'] = train['Sentence'].apply(lambda x : x.strip())
dev['Sentence'] = dev['Sentence'].apply(lambda x: x.strip())
test['Sentence'] = test['Sentence'].apply(lambda x:x.strip())
train.head()
dev.head()

,id,Sentence,Label
0,TR-0,What is here called controlled natural languag...,O O O O B-long I-long I-long O B-short O O O O...
1,TR-1,"In this paper , we introduce CoQA , a Conversa...",O O O O O O B-short O O B-long I-long I-long O...
2,TR-2,"in : Proceedings of SODA 98 , Ninth Annual ACM...",O O O O B-short O O O O B-short O B-short O O ...
3,TR-3,But is the existing HPC I / O stack still suit...,O O O O B-short I-short I-short I-short O O O ...
4,TR-4,"For a fixed and known , we apply a successive ...",O O O O O O O O O B-long I-long I-long O B-sho...


,id,Sentence,Label
0,DEV-0,The second case in which it helps to pose a no...,O O O O O O O O O O O O B-short O O B-short O ...
1,DEV-1,We created such datasets for each kind of Ques...,O O O O O O O O B-long I-long O O O O O O O O ...
2,DEV-2,"Related Work Currently , several researchers s...",O O O O O O O O O O O O O O B-short O O
3,DEV-3,"However , we observed a decrease in the number...",O O O O O O O O O O O O O O O O O O O O O O O ...
4,DEV-4,"Here , DA = direct assessment , RR = relative ...",O O B-short O B-long I-long O B-short O B-long...


In [32]:
def label_replacer(x):
  x = x.replace('B-short','short')
  x = x.replace('I-short','short')
  x = x.replace('B-long','long')
  x = x.replace('I-long','long')
  return x

In [35]:
train['Label'] = train['Label'].apply(label_replacer)
dev['Label'] = dev['Label'].apply(label_replacer)
test['Label'] = test['Label'].apply(label_replacer)
train.head()
dev.head()

,id,Sentence,Label,POS
0,TR-0,What is here called controlled natural languag...,O O O O long long long O short O O O O O O O O O,PRON AUX ADV VERB VERB ADJ NOUN PUNCT PROPN PU...
1,TR-1,"In this paper , we introduce CoQA , a Conversa...",O O O O O O short O O long long long O O O O O...,ADP DET NOUN PUNCT PRON VERB PROPN PUNCT DET P...
2,TR-2,"in : Proceedings of SODA 98 , Ninth Annual ACM...",O O O O short O O O O short O short O O O O O ...,ADP PUNCT NOUN ADP NOUN NUM PUNCT PROPN PROPN ...
3,TR-3,But is the existing HPC I / O stack still suit...,O O O O short short short short O O O O O shor...,CCONJ AUX DET VERB NOUN PRON SYM INTJ ADV ADV ...
4,TR-4,"For a fixed and known , we apply a successive ...",O O O O O O O O O long long long O short O O O...,ADP DET VERB CCONJ VERB PUNCT PRON VERB DET AD...


,id,Sentence,Label,POS
0,DEV-0,The second case in which it helps to pose a no...,O O O O O O O O O O O O short O O short O O O ...,DET ADJ NOUN ADP DET PRON VERB PART VERB DET A...
1,DEV-1,We created such datasets for each kind of Ques...,O O O O O O O O long long O O O O O O O O O O ...,PRON VERB ADJ NOUN ADP DET NOUN ADP NOUN PROPN...
2,DEV-2,"Related Work Currently , several researchers s...",O O O O O O O O O O O O O O short O O,ADJ NOUN ADV PUNCT ADJ NOUN VERB NOUN ADP DET ...
3,DEV-3,"However , we observed a decrease in the number...",O O O O O O O O O O O O O O O O O O O O O O O ...,ADV PUNCT PRON VERB DET NOUN ADP DET NOUN ADP ...
4,DEV-4,"Here , DA = direct assessment , RR = relative ...",O O short O long long O short O long long O sh...,ADV PUNCT PROPN SYM ADJ NOUN PUNCT PROPN SYM A...


In [18]:
from tqdm import tqdm_notebook as tqdm
def pos_tagger(df):
  df_pos_tags = []
  for i, val in tqdm(df.iterrows()):
    doc = nlp(df['Sentence'][i])
    pos_tags = []
    for token in doc:
      pos_tags.append(token.pos_)
    pos_tags = ' '.join(pos_tags)
    df_pos_tags.append(pos_tags)
  df['POS']= df_pos_tags
pos_tagger(train)
pos_tagger(dev)
pos_tagger(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [36]:
train.head()

,id,Sentence,Label,POS
0,TR-0,What is here called controlled natural languag...,O O O O long long long O short O O O O O O O O O,PRON AUX ADV VERB VERB ADJ NOUN PUNCT PROPN PU...
1,TR-1,"In this paper , we introduce CoQA , a Conversa...",O O O O O O short O O long long long O O O O O...,ADP DET NOUN PUNCT PRON VERB PROPN PUNCT DET P...
2,TR-2,"in : Proceedings of SODA 98 , Ninth Annual ACM...",O O O O short O O O O short O short O O O O O ...,ADP PUNCT NOUN ADP NOUN NUM PUNCT PROPN PROPN ...
3,TR-3,But is the existing HPC I / O stack still suit...,O O O O short short short short O O O O O shor...,CCONJ AUX DET VERB NOUN PRON SYM INTJ ADV ADV ...
4,TR-4,"For a fixed and known , we apply a successive ...",O O O O O O O O O long long long O short O O O...,ADP DET VERB CCONJ VERB PUNCT PRON VERB DET AD...


In [37]:
test['Label'] = test['Sentence'].apply(lambda x: ' '.join(['O']*len(x.split())))
test['Label']

0       O O O O O O O O O O O O O O O O O O O O O O O ...
1       O O O O O O O O O O O O O O O O O O O O O O O ...
2                       O O O O O O O O O O O O O O O O O
3       O O O O O O O O O O O O O O O O O O O O O O O ...
4             O O O O O O O O O O O O O O O O O O O O O O
                              ...                        
1712    O O O O O O O O O O O O O O O O O O O O O O O ...
1713                            O O O O O O O O O O O O O
1714    O O O O O O O O O O O O O O O O O O O O O O O ...
1715    O O O O O O O O O O O O O O O O O O O O O O O ...
1716    O O O O O O O O O O O O O O O O O O O O O O O ...
Name: Label, Length: 1717, dtype: object

In [38]:
def data_prep(df):
  data = []
  for i in range(df.shape[0]):
      # Take the word, POS tag, and its label
      words= df['Sentence'][i].split()
      pos =df['POS'][i].split()
      label = df['Label'][i].split()
      data.append([(w, pos, label) for (w, pos, label) in zip(words, pos, label)])
  return data
train_prep = data_prep(train)
dev_prep = data_prep(dev)
test_prep = data_prep(test)

In [39]:
train_prep[0]

[('What', 'PRON', 'O'),
 ('is', 'AUX', 'O'),
 ('here', 'ADV', 'O'),
 ('called', 'VERB', 'O'),
 ('controlled', 'VERB', 'long'),
 ('natural', 'ADJ', 'long'),
 ('language', 'NOUN', 'long'),
 ('(', 'PUNCT', 'O'),
 ('CNL', 'PROPN', 'short'),
 (')', 'PUNCT', 'O'),
 ('has', 'AUX', 'O'),
 ('traditionally', 'ADV', 'O'),
 ('been', 'AUX', 'O'),
 ('given', 'VERB', 'O'),
 ('many', 'ADJ', 'O'),
 ('different', 'ADJ', 'O'),
 ('names', 'NOUN', 'O'),
 ('.', 'PUNCT', 'O')]

In [40]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_prep]
y_train = [sent2labels(s) for s in train_prep]

X_test = [sent2features(s) for s in dev_prep]
y_test = [sent2labels(s) for s in dev_prep]

In [41]:
X_train[0][1]

{'+1:postag': 'ADV',
 '+1:postag[:2]': 'AD',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'here',
 '-1:postag': 'PRON',
 '-1:postag[:2]': 'PR',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.lower()': 'what',
 'bias': 1.0,
 'postag': 'AUX',
 'postag[:2]': 'AU',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': 'is',
 'word[-3:]': 'is'}

In [42]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 27.2 s, sys: 32.8 ms, total: 27.2 s
Wall time: 27.2 s


In [ ]:
# eli5.show_weights(crf, top=30)

In [43]:
labels = list(crf.classes_)
labels.remove('O')
labels

['long', 'short']

In [47]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='macro', labels=labels)

0.8871135937513808

In [48]:
X_test_final = [sent2features(s) for s in test_prep]
y_test_final = [sent2labels(s) for s in test_prep]

In [49]:
y_pred_final = crf.predict(X_test_final)

In [50]:
test['Label'] =  pd.DataFrame([' '.join(x) for x in y_pred_final])
test.head()


,id,Sentence,Label,POS
0,DEV-0,The second case in which it helps to pose a no...,O O O O O O O O O O O O O O O short O O O O O ...,DET ADJ NOUN ADP DET PRON VERB PART VERB DET A...
1,DEV-1,We created such datasets for each kind of Ques...,O O O O O O O O O O O O O O O O O O O O O O O ...,PRON VERB ADJ NOUN ADP DET NOUN ADP NOUN PROPN...
2,DEV-2,"Related Work Currently , several researchers s...",O O O O O O O O O O O O O O short O O,ADJ NOUN ADV PUNCT ADJ NOUN VERB NOUN ADP DET ...
3,DEV-3,"However , we observed a decrease in the number...",O O O O O O O O O O O O O O O O O O O O O O O ...,ADV PUNCT PRON VERB DET NOUN ADP DET NOUN ADP ...
4,DEV-4,"Here , DA = direct assessment , RR = relative ...",O O short O long long O short O long long O sh...,ADV PUNCT PROPN SYM ADJ NOUN PUNCT PROPN SYM A...


In [ ]:
test[['Sentence','Label']].to_csv('submit_task2_test.csv', index=False)

In [51]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       short      0.951     0.880     0.914      3338
        long      0.878     0.844     0.860      5082

   micro avg      0.906     0.858     0.881      8420
   macro avg      0.914     0.862     0.887      8420
weighted avg      0.907     0.858     0.882      8420



## Results

Baseline to beat - 0.8546 

Current using CRF - 0.809
```
precision    recall  f1-score   support

      B-long      0.842     0.792     0.817      1613
      I-long      0.885     0.856     0.870      3469
     B-short      0.943     0.899     0.920      3147
     I-short      0.748     0.545     0.630       191

   micro avg      0.896     0.852     0.874      8420
   macro avg      0.855     0.773     0.809      8420
weighted avg      0.895     0.852     0.873      8420

```
Using CRF with modified labels - 0.887

```
              precision    recall  f1-score   support

       short      0.951     0.880     0.914      3338
        long      0.878     0.844     0.860      5082

   micro avg      0.906     0.858     0.881      8420
   macro avg      0.914     0.862     0.887      8420
weighted avg      0.907     0.858     0.882      8420

```

# For later - Hyperparameter tuning and model analysis

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=200, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

                          precision    recall  f1-score   support

         B-Alias-Te-frag      0.000     0.000     0.000         0
            B-Alias-Term      0.000     0.000     0.000        20
            I-Alias-Term      0.000     0.000     0.000        25
         B-Definiti-frag      0.000     0.000     0.000         1
         I-Definiti-frag      0.000     0.000     0.000         2
            B-Definition      0.000     0.000     0.000       288
            I-Definition      0.536     0.549     0.542      4114
             B-Qualifier      0.000     0.000     0.000         3
             I-Qualifier      0.000     0.000     0.000        22
B-Referential-Definition      0.000     0.000     0.000         2
I-Referential-Definition      0.000     0.000     0.000         2
      B-Referential-Term      0.000     0.000     0.000         3
      I-Referential-Term      0.000     0.000     0.000         1
  B-Secondary-Definition      0.000     0.000     0.000         2
  I-Secon

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

y_pred = crf.predict(X_test)
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

                          precision    recall  f1-score   support

         B-Alias-Te-frag      0.000     0.000     0.000         0
            B-Alias-Term      0.000     0.000     0.000        20
            I-Alias-Term      0.000     0.000     0.000        25
         B-Definiti-frag      0.000     0.000     0.000         1
         I-Definiti-frag      0.000     0.000     0.000         2
            B-Definition      0.000     0.000     0.000       288
            I-Definition      0.527     0.579     0.552      4114
             B-Qualifier      0.000     0.000     0.000         3
             I-Qualifier      0.000     0.000     0.000        22
B-Referential-Definition      0.000     0.000     0.000         2
I-Referential-Definition      0.000     0.000     0.000         2
      B-Referential-Term      0.000     0.000     0.000         3
      I-Referential-Term      0.000     0.000     0.000         1
  B-Secondary-Definition      0.000     0.000     0.000         2
  I-Secon

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
eli5.show_weights(crf, top=10, targets=['O', 'B-ORG', 'I-ORG'])

In [ ]:
expl = eli5.explain_weights(crf, top=5, targets=['O', 'B-LOC', 'I-LOC'])
print(eli5.format_as_text(expl))

In [ ]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=5,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed: 42.1min
[Parallel(n_jobs=5)]: Done 150 out of 150 | elapsed: 149.0min finished


CPU times: user 58min 56s, sys: 34.5 s, total: 59min 30s
Wall time: 2h 30min 30s


In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 1.6483342862953896, 'c2': 0.12940114154937338}
best CV score: 0.7180427900104681
model size: 0.61M


In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                          precision    recall  f1-score   support

         B-Alias-Te-frag      0.000     0.000     0.000         0
            B-Alias-Term      0.700     0.350     0.467        20
            I-Alias-Term      1.000     0.040     0.077        25
         B-Definiti-frag      0.000     0.000     0.000         1
         I-Definiti-frag      0.000     0.000     0.000         2
            B-Definition      0.619     0.552     0.583       288
            I-Definition      0.748     0.891     0.813      4114
             B-Qualifier      0.000     0.000     0.000         3
             I-Qualifier      0.000     0.000     0.000        22
B-Referential-Definition      0.000     0.000     0.000         2
I-Referential-Definition      0.000     0.000     0.000         2
      B-Referential-Term      0.000     0.000     0.000         3
      I-Referential-Term      0.000     0.000     0.000         1
  B-Secondary-Definition      0.000     0.000     0.000         2
  I-Secon

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])